In [8]:
import numpy as np

In [35]:
# set dimension
n = 2
d = 3

# set lambda
lamb = 10

# generate random X and y
X = np.random.randn(n, d) + 40
y = np.random.binomial(1, 0.5, (n, 1))

# compute Q, p, A and b
Q = - 0.5 * np.linalg.inv(X.T.dot(X))
p = - np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y))
A = np.concatenate([np.eye(d), -np.eye(d)], axis=0)
b = lamb * np.ones((2 * d, 1))

# possible starting point
v0 = (lamb / 2) * np.ones((d, 1))

In [10]:
x = np.random.randn(d, 1)

In [11]:
# xx_1 = grad_f(Q, p, A, b, x)
# xx_2 = hess_f(Q, A, b, x)

In [79]:
def f(Q, p, A, b, x):
    phi = 0
    for i in range(np.shape(A)[0]):
        if - float(A[i, :].dot(x) - b[i, 0]) > 0:
            phi -= np.log(- float(A[i, :].dot(x) - b[i, 0]))
        else:
            return np.inf
    return float(x.T.dot(Q.dot(x))) + float(p.T.dot(x)) + phi

def grad_f(Q, p, A, b, x):
    phi = np.zeros_like(A[0, :])
    for i in range(np.shape(A)[0]):
        phi += - (1 / (A[i, :].dot(x) - b[i, 0])) * A[i, :].T
    return (Q.T + Q).dot(x) + p + phi.reshape(len(phi), 1)
        
def hess_f(Q, A, b, x):
    phi = np.zeros_like(np.outer(A[0, :], A[0, :]))
    for i in range(np.shape(A)[0]):
        phi +=  (1 / (A[i, :].dot(x) - b[i, 0]) ** 2) * np.outer(A[i, :], A[i, :])
    return Q + Q.T + phi

In [84]:
def backtracking_line_search(Q, p, A, b, x, delta, grad_f):
    alpha, beta, t, k = 0.1, 0.9, 1, 0
    while f(Q, p, A, b, x + t * delta) >= f(Q, p, A, b, x) + alpha * t * float(grad_f.T.dot(delta)) and k < 1e10:
        t *= beta
        k += 1
        #print(f(Q, p, A, b, x + t * delta), f(Q, p, A, b, x) + alpha * t * float(grad_f.T.dot(delta)))
    return t

def centeringstep(Q, p, A, b, t, v0, eps):
    v_seq = [v0]
    decrement = 1e5
    while decrement / 2 > eps:
        grad = grad_f(t * Q, t * p, A, b, v_seq[-1])
        delta = - np.linalg.inv(hess_f(t * Q, A, b, v_seq[-1])).dot(grad) # newton step
        decrement = grad.T.dot(- delta) # decrement
        gamma = backtracking_line_search(Q, p, A, b, v_seq[-1], delta, grad) # backtrackling line search
        v_seq.append(v_seq[-1] + gamma * delta)
    print(gamma)
    return v_seq
    
def barrmethod(Q, p, A, b, v0, eps):
    m, v_seq = np.shape(A)[0], [v0]
    t, mu = 1, 10
    while m / t >= eps:
        v_seq.append(centeringstep(Q, p, A, b, t, v_seq[-1], eps)[-1]) # minimize t * f_0 + phi and update v_seq (x_star)
        t *= mu
        #print( m / t)
    return v_seq

In [77]:
v_seq = centeringstep(Q, p, A, b, 10000, v_seq[-1], 1e-15)

In [78]:
v_seq[-1]

array([[9.67286864],
       [9.95169845],
       [9.99999995]])

In [70]:
v_seq

[array([[5.],
        [5.],
        [5.]]), array([[9.67343105],
        [9.90537266],
        [9.92331289]])]

In [85]:
v_seq = barrmethod(Q, p, A, b, v0, 1e-15)
v_seq[-1]

[[5.]
 [5.]
 [5.]] [[-2.47412109]
 [-2.12451172]
 [-1.94042969]] [[2.52587891]
 [2.87548828]
 [3.05957031]]
9.242298322399366e-13
[[5.]
 [5.]
 [5.]] [[5.90820312]
 [6.41601562]
 [6.5625    ]] [[10.90820312]
 [11.41601562]
 [11.5625    ]]
0.7290000000000001
[[9.30708008]
 [9.67727539]
 [9.7840625 ]] [[-0.1673584 ]
 [ 0.03283691]
 [ 0.14093018]] [[9.13972168]
 [9.7101123 ]
 [9.92499268]]
1
[[9.13972168]
 [9.7101123 ]
 [9.92499268]] [[0.36259032]
 [0.43616151]
 [0.47789767]] [[ 9.502312  ]
 [10.14627382]
 [10.40289034]]
0.1500946352969992
[[9.19414454]
 [9.77557781]
 [9.99672255]] [[-0.11383283]
 [-0.04109076]
 [ 0.00829093]] [[ 9.08031171]
 [ 9.73448705]
 [10.00501348]]
0.38742048900000015
[[9.15004337]
 [9.75965841]
 [9.99993463]] [[-1.10861168e-01]
 [-4.49129937e-02]
 [-1.07569185e-05]] [[9.0391822 ]
 [9.71474541]
 [9.99992387]]
1
[[9.0391822 ]
 [9.71474541]
 [9.99992387]] [[0.04693097]
 [0.01938492]
 [0.00061795]] [[ 9.08611317]
 [ 9.73413033]
 [10.00054182]]
0.10941898913151243
[[9.0

array([[ 9.11502209],
       [ 9.74551282],
       [10.        ]])

In [81]:
from sklearn import linear_model

clf = linear_model.Lasso(alpha=lamb, fit_intercept=False)
clf.fit(X, y)

print(clf.coef_)

[0.         0.01873852 0.        ]


In [115]:
def phi(A, b, x):
    if (b - A.dot(x) > 0).all():
        return - np.log(b - A.dot(x)).sum() # phi(x) = - sum log(-f_i(x))
    else:
        return np.inf
#         print(x)
#         raise Exception('problem in log') 

def grad_phi(A, b, x):
    res = np.zeros_like(A[0, :])
    for i in range(np.shape(A)[0]):
          res += - (1 / (A[i, :].dot(x) - b[i])) * A[i, :].T
    return np.reshape(res, (np.shape(A)[1], 1))

def hess_phi(A, b, x):
    res = np.zeros((np.shape(A)[1], np.shape(A)[1]))
    for i in range(np.shape(A)[0]):
          res +=  ((1 / (A[i, :].dot(x) - b[i])) ** 2) * np.outer(A[i, :], A[i, :])
    return res

def f(Q, p, A, b, x):
    return x.T.dot(Q.dot(x)) + p.T.dot(x) + phi(A, b, x)
    
def grad(Q, p, A, b, x):
    return (Q.T + Q).dot(x) + p + grad_phi(A, b, x)
    
def hess(Q, p, A, b, x):
    return Q.T + Q + hess_phi(A, b, x)

def backtracking_line_search(Q, p, A, b, x, delta_x):
    alpha, beta, step, k = 0.1, 0.99, 1, 0
    while f(Q, p, A, b, x + step * delta_x) >= f(Q, p, A, b, x) + alpha * step * grad(Q, p, A, b, x).T.dot(delta_x) and k < 1e10:
        step *= beta
        k += 1
    return step # if k < 1e10 else 1
    
# def newton(Q, p, A, b, x):
#     k, max_iter = 0, 10000
#     while np.linalg.norm(grad(Q, p, A, b, x), ord=2) > 1e-6 and k < max_iter:
#         #print(np.linalg.norm(grad(Q, p, A, b, x), ord=2))
#         delta_x = - np.linalg.inv(hess(Q, p, A, b, x)).dot(grad(Q, p, A, b, x))
#         step_size = 1 #backtracking_line_search(Q, p, A, b, x, delta_x)
#         x += step_size * delta_x
#         k += 1
#     return x

# def barrier(Q, p, A, b, t, v0, eps):
#     m = np.shape(A)[0]
#     v = [v0]
#     mu = 20 
#     k, max_iter = 0, 1e10
#     while k < max_iter:
#         v.append(newton(t * Q, t * p, A, b, v[-1])) # minimize t * f_0 + phi and update v
#         if m / t < eps: # stopping criterion 
#             return v
#         t *= mu # increase t
#         k += 1 # increase k
#         print(k)

In [111]:
v = barrier(Q, p, A, b, 1, v0, 1e-5)

NameError: name 'barrier' is not defined

In [ ]:
v[-1]

In [ ]:
from sklearn import linear_model

clf = linear_model.Lasso(alpha=lamb, fit_intercept=False)
clf.fit(X, y)

print(clf.coef_)